# Аналитика ДТП по предоставленным данным

Заказчиком (Оунером задачи) выступает проект «Карта ДТП» https://dtp-stat.ru/ — некоммерческий проект, посвященный проблеме дорожно-транспортных происшествий в России. Это платформа сбора данных о ДТП, бесплатный и открытый сервис аналитики ДТП.

**Цель проекта** - провести глубокий анализ данных, сформулировать и проверить гипотезы, опираясь на доступные признаки.

**Задачи**
* Провести исследовательский анализ данных о ДТП за 2025 год.
* Сформулировать и проверить не менее трех гипотез, основываясь на имеющихся признаках.
* Провести анализ ДТП по всем датасетам, представленным на сайте.
* Построить дашборд используя любой удобный инструмент, с учетом того, что дашборд может быть опубликован

**Декомпозиция работы:**
* Загрузка датасета и знакомство с данными
* Предобработка данных
* Исследовательский анализ данных
* Проверка гипотез
* Общие выводы
* Подготовка дашборда

## 1. Загрузка датасета и знакомство с данными

Загрузим необходимые библиотеки:

In [17]:
#!pip install geopandas

import os
import geopandas as gpd
import pandas as pd
from shapely import wkt

# Настройка отображения Jupiter
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None) # вывод результатов без сокращения количества столбцов

**Для скачивания датасетов с сайта - необходимо использовать скрипт из ноутбука "DataLoader.ipynb"**

Загрузим данные:

In [18]:
all_records = []

DOWNLOAD_DIR = 'geojson_files'

# перебираем все .geojson в папке
for filename in os.listdir(DOWNLOAD_DIR):
    if filename.endswith(".geojson"):
        path = os.path.join(DOWNLOAD_DIR, filename)
        try:
            with open(path, encoding="utf-8") as f:
                data = json.load(f)
            accidents = data["features"]

            records = []
            for feature in accidents:
                props = feature["properties"].copy()
                props["geometry"] = feature["geometry"]
                records.append(props)

            all_records.extend(records)
            print(f"Загружено {len(records)} строк из {filename}")

        except Exception as e:
            print(f"Ошибка при обработке {filename}: {e}")

# объединяем в один датафрейм
data = pd.DataFrame(all_records)
print(f"Итого строк во всех файлах: {len(data)}")

Загружено 10307 строк из novgorodskaia-oblast.geojson
Загружено 27892 строк из tiumenskaia-oblast.geojson
Загружено 5855 строк из sevastopol.geojson
Загружено 41694 строк из cheliabinskaia-oblast.geojson
Загружено 7015 строк из respublika-kareliia.geojson
Загружено 25787 строк из volgogradskaia-oblast.geojson
Загружено 20977 строк из vladimirskaia-oblast.geojson
Загружено 2323 строк из magadanskaia-oblast.geojson
Загружено 10181 строк из kurganskaia-oblast.geojson
Загружено 29859 строк из sverdlovskaia-oblast.geojson
Загружено 16533 строк из respublika-dagestan.geojson
Загружено 5274 строк из respublika-tyva.geojson
Загружено 2589 строк из chechenskaia-respublika.geojson
Загружено 12534 строк из arkhangelskaia-oblast.geojson
Загружено 5254 строк из respublika-kalmykiia.geojson
Загружено 13438 строк из tambovskaia-oblast.geojson
Загружено 28453 строк из omskaia-oblast.geojson
Загружено 6748 строк из kabardino-balkarskaia-respublika.geojson
Загружено 4274 строк из iamalo-nenetskii-avtono

Откроем датасет и посмотрим на данные:

In [19]:
data.head(5)

,id,tags,light,point,nearby,region,scheme,address,weather,category,datetime,severity,vehicles,dead_count,participants,injured_count,parent_region,road_conditions,participants_count,participant_categories,geometry
0,1504730,[Дорожно-транспортные происшествия],Светлое время суток,"{'lat': 58.332904, 'long': 30.415138}",[],Шимский район,200,"Медведь-Батецкий, 7 км",[Ясно],Столкновение,2025-05-01 12:55:00,С погибшими,"[{'year': 2012, 'brand': 'KIA', 'color': 'Серы...",1,[],0,Новгородская область,[Сухое],3,"[Все участники, Мотоциклисты]","{'type': 'Point', 'coordinates': [30.415138, 5..."
1,156565,[Дорожно-транспортные происшествия],Светлое время суток,"{'lat': None, 'long': None}",[],Шимский район,600,"Батецкий-Медведь, 10 км",[Ясно],Опрокидывание,2015-06-11 16:10:00,Легкий,"[{'year': 2011, 'brand': 'KIA', 'color': 'Сини...",0,[],1,Новгородская область,[Сухое],1,[Все участники],"{'type': 'Point', 'coordinates': [None, None]}"
2,156274,[Дорожно-транспортные происшествия],"В темное время суток, освещение включено","{'lat': 58.295429, 'long': 30.494614}","[Жилые дома индивидуальной застройки, Нерегули...",Шимский район,840,"д Старый Медведь, Медведь-Батецкий, 1 км",[Ясно],Наезд на пешехода,2022-10-07 20:30:00,Тяжёлый,"[{'year': 2009, 'brand': 'ВАЗ', 'color': 'Черн...",0,"[{'role': 'Пешеход', 'gender': 'Мужской', 'vio...",1,Новгородская область,[Сухое],2,"[Все участники, Пешеходы]","{'type': 'Point', 'coordinates': [30.494614, 5..."
3,156238,[Дорожно-транспортные происшествия],"В темное время суток, освещение включено","{'lat': 58.210136, 'long': 30.721529}","[Административные здания, Многоквартирные жилы...",Шимский район,740,"рп Шимск, ул Новгородская, 6",[Пасмурно],Наезд на пешехода,2022-12-06 17:05:00,Легкий,"[{'year': 2009, 'brand': 'CHEVROLET', 'color':...",0,"[{'role': 'Пешеход', 'gender': 'Женский', 'vio...",1,Новгородская область,[Сухое],2,"[Все участники, Пешеходы]","{'type': 'Point', 'coordinates': [30.721529, 5..."
4,156240,[Дорожно-транспортные происшествия],Светлое время суток,"{'lat': 58.067255, 'long': 30.765452}",[],Шимский район,610,"Шимск-Волот, 14 км",[Снегопад],Съезд с дороги,2021-11-27 15:00:00,Легкий,"[{'year': 2010, 'brand': 'ПАЗ', 'color': 'Белы...",0,[],1,Новгородская область,"[Недостатки зимнего содержания, Заснеженное]",3,[Все участники],"{'type': 'Point', 'coordinates': [30.765452, 5..."


Посмотрим общую информацию о датасете:

In [ ]:
data.info()

In [ ]:
print(f'Исходный датасет содержит {data.shape[0]} записей и {data.shape[1]} признак(а/ов):')

* `id`: идентификатор
* `light`: время суток
* `point`: координаты (гео)
* `nearby`: комментарий по месту происшествия (где произошло)
* `region`: город/район
* `address`: адрес
* `weather`: погода
* `category`: тип ДТП
* `datetime`: дата и время происшествия
* `severity`: тяжесть ДТП/вред здоровью
* `vehicles`: участники – транспортные средства:
    * `year`: год производства транспортного средства
    * `brand`: марка транспортного средства
    * `color`: цвет транспортного средства
    * `model`: модель транспортного средства
    * `category`: категория транспортного средства
    * `role`: роль участника
    * `gender`: пол участника
    * `violations`: нарушения правил участником
    * `health_status`: состояние здоровья участника
    * `years_of_driving_experience`: стаж вождения участника (только у водителей)
* `dead_count`: кол-во погибших в ДТП
* `participants`: участники без транспортных средств (описание, как у участников внутри транспортных средств)
* `injured_count`: кол-во раненых в ДТП
* `parent_region`: родительский регион
* `road_conditions`: состояние дорожного покрытия
* `participants_count`: кол-во участников ДТП
* `participant_categories`: категории участников

## 2. Предобработка данных

Задачи данного раздела:

* Привести типы данных в соответствие
* Проверить пропуски в датасете и обработать их
* Проверить наличие явных и неявных дубликатов и обработать их
* По возможности дообогатить данные на основе имеющихся

### 2.1. Обработка пропусков данных

Проверим наличие NAN и NULL в датасете:

In [ ]:
print(20*'==')
print('NULLS')
print(data.isnull().sum())
print(20*'==')
print('NANs')
print(data.isna().sum())
print(20*'==')

Обнаружены пропуски в трех полях - `scheme` (данное поле не указано в описании данных), `address`, `geometry`.

Проверим какие данные содержатся в поле `scheme`:

In [ ]:
data['scheme'].unique()

Здесь в основном представлены трехзначные числа в формате строки. Заполним пропуски заглушкой "unknown"

In [ ]:
data['scheme'] = data['scheme'].fillna('unknown')

Пропуски по полю `address` также заполним заглушкой:

In [ ]:
data['address'] = data['address'].fillna('unknown')

### 2.2. Подготовка полей

#### 2.2.1 Преобразование списочных полей к текстовому типу

В полях `tags`, `nearby`, `weather`, `vehicles`, `participants`, `road_conditions`, `participant_categories` содержатся списки, преобразуем данные поля к текстовому типу.

Определим перечень полей для преобразования:

In [ ]:
list_features = ['tags', 'nearby', 'weather', 'road_conditions', 'participant_categories']

Приведем поля к текстовому типу:

In [21]:
for column in list_features:
    data[column] = data[column].apply(lambda x: x[0] if len(x) == 1 else ', '.join(x) if isinstance(x, list) else '')

Проверим результат:

In [22]:
data.head()

,id,tags,light,point,nearby,region,scheme,address,weather,category,datetime,severity,vehicles,dead_count,participants,injured_count,parent_region,road_conditions,participants_count,participant_categories,geometry
0,1504730,Дорожно-транспортные происшествия,Светлое время суток,"{'lat': 58.332904, 'long': 30.415138}",,Шимский район,200,"Медведь-Батецкий, 7 км",Ясно,Столкновение,2025-05-01 12:55:00,С погибшими,"[{'year': 2012, 'brand': 'KIA', 'color': 'Серы...",1,[],0,Новгородская область,Сухое,3,"Все участники, Мотоциклисты","{'type': 'Point', 'coordinates': [30.415138, 5..."
1,156565,Дорожно-транспортные происшествия,Светлое время суток,"{'lat': None, 'long': None}",,Шимский район,600,"Батецкий-Медведь, 10 км",Ясно,Опрокидывание,2015-06-11 16:10:00,Легкий,"[{'year': 2011, 'brand': 'KIA', 'color': 'Сини...",0,[],1,Новгородская область,Сухое,1,Все участники,"{'type': 'Point', 'coordinates': [None, None]}"
2,156274,Дорожно-транспортные происшествия,"В темное время суток, освещение включено","{'lat': 58.295429, 'long': 30.494614}","Жилые дома индивидуальной застройки, Нерегулир...",Шимский район,840,"д Старый Медведь, Медведь-Батецкий, 1 км",Ясно,Наезд на пешехода,2022-10-07 20:30:00,Тяжёлый,"[{'year': 2009, 'brand': 'ВАЗ', 'color': 'Черн...",0,"[{'role': 'Пешеход', 'gender': 'Мужской', 'vio...",1,Новгородская область,Сухое,2,"Все участники, Пешеходы","{'type': 'Point', 'coordinates': [30.494614, 5..."
3,156238,Дорожно-транспортные происшествия,"В темное время суток, освещение включено","{'lat': 58.210136, 'long': 30.721529}","Административные здания, Многоквартирные жилые...",Шимский район,740,"рп Шимск, ул Новгородская, 6",Пасмурно,Наезд на пешехода,2022-12-06 17:05:00,Легкий,"[{'year': 2009, 'brand': 'CHEVROLET', 'color':...",0,"[{'role': 'Пешеход', 'gender': 'Женский', 'vio...",1,Новгородская область,Сухое,2,"Все участники, Пешеходы","{'type': 'Point', 'coordinates': [30.721529, 5..."
4,156240,Дорожно-транспортные происшествия,Светлое время суток,"{'lat': 58.067255, 'long': 30.765452}",,Шимский район,610,"Шимск-Волот, 14 км",Снегопад,Съезд с дороги,2021-11-27 15:00:00,Легкий,"[{'year': 2010, 'brand': 'ПАЗ', 'color': 'Белы...",0,[],1,Новгородская область,"Недостатки зимнего содержания, Заснеженное",3,Все участники,"{'type': 'Point', 'coordinates': [30.765452, 5..."


Проверим уникальные значения по полю `tags`:

In [23]:
data['tags'].unique()

array(['Дорожно-транспортные происшествия'], dtype=object)

In [ ]:
Данное поле не несет смысловой нагрузк

In [ ]:
'', 'nearby', 'weather', 'road_conditions', 'participant_categories'

In [21]:
data.columns.tolist()

['id',
 'tags',
 'light',
 'point',
 'nearby',
 'region',
 'scheme',
 'address',
 'weather',
 'category',
 'datetime',
 'severity',
 'vehicles',
 'dead_count',
 'participants',
 'injured_count',
 'parent_region',
 'road_conditions',
 'participants_count',
 'participant_categories',
 'geometry']

### 2.3. Обработка дубликатов

Проверим количество дубликатов в датасете:

In [20]:
print(data.dtypes)

id                         int64
tags                      object
light                     object
point                     object
nearby                    object
region                    object
scheme                    object
address                   object
weather                   object
category                  object
datetime                  object
severity                  object
vehicles                  object
dead_count                 int64
participants              object
injured_count              int64
parent_region             object
road_conditions           object
participants_count         int64
participant_categories    object
geometry                  object
dtype: object


In [19]:
data.duplicated().sum()

TypeError: unhashable type: 'list'

Явных дубликатов не обнаружено.

Проверим наличие неявных дубликатов по категорийным полям.

Проверим уникальные значения по полю `light`:

In [21]:
data['light'].unique().tolist()

['В темное время суток, освещение включено',
 'Светлое время суток',
 'В темное время суток, освещение отсутствует',
 'Сумерки',
 'В темное время суток, освещение не включено',
 'Не установлено']

Неявных дубликатов по полю `light` не обнаружено.

### 2.3. Дообогащение данных

#### 2.3.1. Добавление координат ДТП, соответствующим формату данных DataLens

Для построения геоточек на картах в DataLens необходимы данные в формате '[55.75222,37.61556]', где первое число - широта, вторая долгота.

Получим сначала отдельно координаты широты и долготы:

In [57]:
data['longitude'] = data['geometry'].geometry.x
data['latitude'] = data['geometry'].geometry.y

In [59]:
data.to_csv('krsk.csv')